In [1]:
import pandas as pd
from datetime import datetime, timedelta

events_df = pd.read_csv('big-data-cup-2025/raw_data/2024-10-13.Team.B.@.Team.A.-.Events.csv')
tracking_df = pd.read_csv('big-data-cup-2025/raw_data/2024-10-13.Team.B.@.Team.A.-.Tracking.csv', low_memory=False)

def time_to_seconds(time_val):
    if isinstance(time_val, str):
        try:
            minutes, seconds = map(float, time_val.split(':'))
            return minutes * 60 + seconds
        except ValueError:
            return None
    elif isinstance(time_val, (int, float)):
        return float(time_val)
    else:
        return None

tracking_df['Game Clock'] = tracking_df['Game Clock'].apply(time_to_seconds)

def synchronize_events_with_tracking(events_df, tracking_df):
    # Find the first event after the start of the period (puck recovery off face-off)
    first_event = events_df[(events_df['Event'] == 'Puck Recovery') &
                            (events_df['Clock'] != '20:00')].iloc[0]

    # Convert event time to seconds
    event_time = datetime.strptime(first_event['Clock'], '%M:%S')
    event_seconds = event_time.minute * 60 + event_time.second

    # Ensure 'Game Clock' is numeric
    tracking_df['Game Clock'] = pd.to_numeric(tracking_df['Game Clock'], errors='coerce')

    # Find the most similar frame in tracking data
    tracking_df['time_diff'] = abs(events_df['Clock'].apply(time_to_seconds) - event_seconds)
    closest_frame = tracking_df.loc[tracking_df['time_diff'].idxmin()]

    # Calculate time difference
    time_diff = closest_frame['Game Clock'] - event_seconds

    # Adjust event times
    def adjust_time(time_str):
        time = datetime.strptime(time_str, '%M:%S')
        new_time = time + timedelta(seconds=time_diff)
        return new_time.strftime('%M:%S')

    events_df['Adjusted_Clock'] = events_df['Clock'].apply(adjust_time)

    return events_df


# Example usage:
synchronized_events = synchronize_events_with_tracking(events_df, tracking_df)

#Create a syncroized events csv file
synchronized_events.to_csv('synchronized_events.csv', index=False)


C:\Users\ryand\AppData\Local\Temp\ipykernel_15036\3125403154.py:8: DtypeWarning: Columns (2,9) have mixed types. Specify dtype option on import or set low_memory=False.
  a_b_tracking = pd.read_csv('raw_data/2024-10-13.Team.B.@.Team.A.-.Tracking.csv')
C:\Users\ryand\AppData\Local\Temp\ipykernel_15036\3125403154.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  c_d_tracking = pd.read_csv('raw_data/2024-11-15.Team.D.@.Team.C.-.Tracking.csv')
C:\Users\ryand\AppData\Local\Temp\ipykernel_15036\3125403154.py:16: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  e_f_tracking = pd.read_csv('raw_data/2024-11-16.Team.F.@.Team.E.-.Tracking.csv')


ValueError: time data 'nan' does not match format '%M:%S.%f'